# Benchmarking Our Algorithm against others

In [6]:
import sys
sys.path.append('../../')
from time import time
import signal
from os import listdir
import numpy as np
import pickle as pkl


from panav.sequential import sequential_planning
from panav.SAMP.solvers import Tube_Planning, Simple_MILP_Planning

from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]


def run_experiments(env_name, 
                    env_info, 
                    N_range, 
                    algs, 
                    TIMEOUT = 100,
                    results = [], 
                    executed = None):
    
    '''
        Checkpointed results and timeouted can be passed in so we can resume operation.
    '''

    if executed is None:
        executed = {alg: {N :False for N in N_range} for alg in algs}

    for N in N_range: 
        # Construct the environment
        constructor = env_info['constructor']
        specs = env_info['specs']
        env = constructor(**specs,N_agent = N)

        # Run the experiment on each algorithm
        for name,alg in algs.items():

            if executed[name][N]:
                continue
            
            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            def timeout_handler(signum,frame):
                print("Algorithm timeout")
                # Mark the rest of the executed table as True, so we don't ever visit this algorithm again
                for n in executed[name]:
                    if n>=N:
                        executed[name][n] = True
                # Save checkpoint data
                with open("./data/{}.pkl".format(env_name),'wb') as fp:
                    pkl.dump({'results':results, 'executed':executed},fp)
                raise TimeoutError("Timeout")
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            # Mark the executed status as True
            executed[name][N] = True

            if t>TIMEOUT:
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                result['plan'] = plan
                print("On time for algorithm", name, "N agent = ", N)

            # Save checkpoint data
            with open("./data/{}.pkl".format(env_name),'wb') as fp:
                pkl.dump({'results':results, 'executed':executed},fp)
            
def get_checkpoint_state(env_name,algs,N_range,first_run = False):
    if first_run:
        results = []
        executed = {alg: {N :False for N in N_range} for alg in algs}
        # Save checkpoint state
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump({'results':results, 'executed':executed},fp)

    # Load checkpoint state
    with open("./data/{}.pkl".format(env_name),'rb') as fp:
        ckpt = pkl.load(fp)

    results = ckpt['results']
    executed = ckpt['executed']

    return results,executed


In [8]:
bloating_r = 0.5
vmax = 1.0

TIMEOUT = 120

# Specify the Algorithms
algs = {  
        "Seq_Tube": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        "Seq_Tube_nonLazy": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),      
        # "Seq_S2M2": lambda env: sequential_planning(Simple_MILP_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),
        "ST-RRT*": lambda env: sequential_ST_RRTStar(env,vmax,bloating_r)
        }
# Load environment presets
env_names = [get_env_name(fn) for fn in listdir('./envs')]

env_info = {}

for name in env_names:
    with open(f'./envs/{name}.pkl','rb') as fp:
        env_info[name] = pkl.load(fp)

In [9]:
env_name = 'Room'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [3,6,9,12,15]

In [10]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Env: Room Alg: Seq_Tube N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube N agent =  3
Env: Room Alg: Seq_Tube_nonLazy N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube_nonLazy N agent =  3
Env: Room Alg: ST-RRT* N agent: 3
Planning for agent 0/3
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 1/3
Info:    SpaceTimeRRT: Created 59 states (18 start + 41 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 2/3Info:    SpaceTimeRRT: Created 49 states (23 start + 26 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


On time for algorithmInfo:    SpaceTimeRRT: Created 170 states (87 start + 83 goal)
 ST-RRT* N agent =  3
Env: Room Alg: Seq_Tube N agent: 6
Planning for agent 0/6
Planning for agent 1/6
Planning for agent 2/6
Planning for agent 3/6
Planning for agent 4/6
Planning for agent 5/6
On time for algorithm Seq_Tube N agent =  6
Env: Room Alg: Seq_Tube_nonLazy N agent: 6
Planning for agent 0/6
Planning for agent 1/6
Planning for agent 2/6
Planning for agent 3/6
Planning for agent 4/6
Planning for agent 5/6
On time for algorithm Seq_Tube_nonLazy N agent =  6
Env: Room Alg: ST-RRT* N agent: 6
Planning for agent 0/6
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


...

Planning for agent 13/15Info:    SpaceTimeRRT: Created 314 states (119 start + 195 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 14/15Info:    SpaceTimeRRT: Created 513 states (174 start + 339 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Increased time bound factor to 4.00
On time for algorithmInfo:    SpaceTimeRRT: Created 703 states (358 start + 345 goal)
 ST-RRT* N agent =  15


In [11]:
env_name = 'MultiTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [3,6,9,12,15]

In [12]:
# results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

In [17]:
env_name = 'SingleTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [3,6,9,12,15]

In [18]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                240,
                results,
                executed)

Env: SingleTunnel Alg: Seq_Tube N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube N agent =  3
Env: SingleTunnel Alg: Seq_Tube_nonLazy N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube_nonLazy N agent =  3
Env: SingleTunnel Alg: ST-RRT* N agent: 3
Planning for agent 0/3
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Info:    SpaceTimeRRT: Created 170 states (75 start + 95 goal)
Planning for agent 1/3
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Increased time bound factor to 4.00
Planning for agent 2/3
Info:    SpaceTimeRRT: Created 579 states (271 start + 308 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


On time for algorithmInfo:    SpaceTimeRRT: Created 372 states (164 start + 208 goal)
 ST-RRT* N agent =  3
Env: SingleTunnel Alg: Seq_Tube N agent: 6
Planning for agent 0/6
Planning for agent 1/6
Planning for agent 2/6
Planning for agent 3/6
Planning for agent 4/6
Planning for agent 5/6
On time for algorithm Seq_Tube N agent =  6
Env: SingleTunnel Alg: Seq_Tube_nonLazy N agent: 6
Planning for agent 0/6
Planning for agent 1/6
Planning for agent 2/6
Planning for agent 3/6
Planning for agent 4/6
Planning for agent 5/6
On time for algorithm Seq_Tube_nonLazy N agent =  6
Env: SingleTunnel Alg: ST-RRT* N agent: 6
Planning for agent 0/6
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 1/6
Info:    SpaceTimeRRT: Created 83 states (36 start + 47 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceT

...

         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Increased time bound factor to 4.00
Planning for agent 11/12Info:    SpaceTimeRRT: Created 1089 states (498 start + 591 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Increased time bound factor to 4.00
On time for algorithmInfo:    SpaceTimeRRT: Created 778 states (357 start + 421 goal)
 ST-RRT* N agent =  12
Env: SingleTunnel Alg: Seq_Tube N agent: 15
Planning for agent 0/15
Planning for agent 1/15
Planning for agent 2/15
Planning for agent 3/15
Planning for agent 4/15
Planning for agent 5/15
Planning for agent 6/15
Planning for agent 7/15
Planning for agent 8/15
Planning for agent 9/15
Planning for agent 10/15
Planning for agent 11/15
Planning for agent 12/15
Planning for agent 13/15
Planning for agent 14/15
On time for algorithm Seq_Tube N agent =  15
Env: SingleTunnel Alg: Seq_Tube_nonLazy N agent: 15
Planning for agent 0/15
Planning for agent 1/15
Planning for agent 2/15
Planning for agent 3/15
Planning for agent 4/15
Planning for agent 5/15
Planning for agent 6/15
Planning for agent 7/15
Planning for agent 8/15
Planning for agent 9/15
Planning for agent 10/15
Planning for agent 11/15
Planning for agent 12/15
Algorithm t

SolverError: Solver 'GUROBI' failed. Try another solver, or solve with verbose=True for more information.

In [15]:
env_name = 'Warehouse'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = [3,6,9,12,15]

In [16]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Env: Warehouse Alg: Seq_Tube N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube N agent =  3
Env: Warehouse Alg: Seq_Tube_nonLazy N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube_nonLazy N agent =  3
Env: Warehouse Alg: ST-RRT* N agent: 3
Planning for agent 0/3
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 1/3
Info:    SpaceTimeRRT: Created 157 states (5 start + 152 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 2/3
Info:    SpaceTimeRRT: Created 38 states (21 start + 17 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
On time for algorithm ST-RRT* N agent =  3
Info:    SpaceTimeRRT: Created 23 states (6 start + 17 goal)
Env: Warehouse Alg: Seq_Tube N agent: 6
Planning for agent 0/6


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 1/6
Planning for agent 2/6
Planning for agent 3/6
Planning for agent 4/6
Planning for agent 5/6
On time for algorithm Seq_Tube N agent =  6
Env: Warehouse Alg: Seq_Tube_nonLazy N agent: 6
Planning for agent 0/6
Planning for agent 1/6
Planning for agent 2/6
Planning for agent 3/6
Planning for agent 4/6
Planning for agent 5/6
On time for algorithm Seq_Tube_nonLazy N agent =  6
Env: Warehouse Alg: ST-RRT* N agent: 6
Planning for agent 0/6
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 1/6
Info:    SpaceTimeRRT: Created 48 states (18 start + 30 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


...

         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 11/15Info:    SpaceTimeRRT: Created 32 states (9 start + 23 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 12/15
Info:    SpaceTimeRRT: Created 22 states (8 start + 14 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 13/15


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Created 47 states (26 start + 21 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 14/15Info:    SpaceTimeRRT: Created 111 states (51 start + 60 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
On time for algorithm ST-RRT* N agent =  15
Info:    SpaceTimeRRT: Created 33 states (14 start + 19 goal)


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
